## Make LD scores for S-LDSC

1000Genomes EUR,
ChIP-seq from ChIP-Atlas

In [1]:
import hail as hl
from hail.linalg import BlockMatrix
hl.init()

Running on Apache Spark version 2.2.3
SparkUI available at http://10.128.0.38:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.12-13681278eb89
LOGGING: writing to /home/hail/hail-20190513-1909-0.2.12-13681278eb89.log


Convenience directories and files

In [117]:
LDSC_dir = "gs://ulirsch_epi511/"

Read in 1000G Matrix table

In [118]:
mt = hl.experimental.load_dataset(name='1000_Genomes_autosomes', version='phase_3', reference_genome='GRCh37')

Filter to EUR and MAF > 0.01

In [119]:
mt = mt.filter_cols(mt.super_population == "EUR")
mt = mt.filter_rows(mt.info['EUR_AF'] >= 0.01)
mt = mt.filter_rows(mt.info['EUR_AF'] <= 0.99)

In [166]:
var = mt.rows().flatten()
var.describe()

----------------------------------------
Global fields:
    'metadata': struct {
        name: str, 
        version: str, 
        reference_genome: str, 
        n_rows: int32, 
        n_cols: int32, 
        n_partitions: int32
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info.CIEND': int32 
    'info.CIPOS': int32 
    'info.CS': str 
    'info.END': int32 
    'info.IMPRECISE': bool 
    'info.MC': array<str> 
    'info.MEINFO': array<str> 
    'info.MEND': int32 
    'info.MLEN': int32 
    'info.MSTART': int32 
    'info.SVLEN': array<int32> 
    'info.SVTYPE': str 
    'info.TSD': str 
    'info.AC': int32 
    'info.AF': float64 
    'info.NS': int32 
    'info.AN': int32 
    'info.EAS_AF': float64 
    'info.EUR_AF': float64 
    'info.AFR_AF': float64 
    'info.AMR_AF': float64 
    'info.SAS_AF': float64 
    'info.DP': int32 
    'inf

Write out filtered Matrix table

In [120]:
mt.write(LDSC_dir + "1KG_EUR_MAF01.mt", overwrite = True)

2019-05-14 05:55:48 Hail: INFO: wrote matrix table with 9789219 rows and 503 columns in 536 partitions to gs://ulirsch_epi511/1KG_EUR_MAF01.mt


In [121]:
mt = hl.read_matrix_table(LDSC_dir + "1KG_EUR_MAF01.mt")

Write out variant IDs

In [167]:
var = mt.rows().flatten().select('locus', 'rsid', 'info.EUR_AF')
var.export(LDSC_dir + "1KG_EUR_MAF01_rsIDs.txt")

2019-05-14 21:05:12 Hail: INFO: while writing:
    gs://ulirsch_epi511/1KG_EUR_MAF01_rsIDs.txt
  merge time: 38.820s


Calculate LD

In [124]:
bm_ld = hl.ld_matrix(mt.GT.n_alt_alleles(), mt.locus, radius = 3e6)
bm_ld = bm_ld ** 2
bm_ld.write(LDSC_dir + "1KG_EUR_MAF01.ld", overwrite = True)

2019-05-14 06:12:18 Hail: INFO: Wrote all 2390 blocks of 9789219 x 503 matrix with block size 4096.
2019-05-14 06:22:39 Hail: INFO: wrote matrix with 9789219 rows and 9789219 columns as 17616 blocks of size 4096 to gs://ulirsch_epi511/1KG_EUR_MAF01.ld


In [126]:
bm_ld = BlockMatrix.read(LDSC_dir + "1KG_EUR_MAF01.ld")

Read in annotation matrix and convert to BlockMatrix

In [127]:
mt_annot = hl.import_matrix_table(LDSC_dir + "1KG_EUR_MAF01_ChIPAtlas_TFs_test.tsv.gz")                

In [129]:
BlockMatrix.write_from_entry_expr(mt_annot.x, LDSC_dir + '1KG_EUR_MAF01_ChIPAtlas_TFs_test.bm', overwrite = True)

2019-05-14 09:20:22 Hail: INFO: Wrote all 2390 blocks of 9789219 x 781 matrix with block size 4096.


In [130]:
bm_annot = BlockMatrix.read(LDSC_dir + "1KG_EUR_MAF01_ChIPAtlas_TFs_test.bm")

Create LD scores

In [131]:
bm_LDscore = bm_ld @ bm_annot
bm_LDscore.write(LDSC_dir + "1KG_EUR_MAF01.ldsc", overwrite = True)

2019-05-14 09:23:19 Hail: INFO: wrote matrix with 9789219 rows and 781 columns as 2390 blocks of size 4096 to gs://ulirsch_epi511/1KG_EUR_MAF01.ldsc


In [132]:
bm_LDscore = BlockMatrix.read(LDSC_dir + "1KG_EUR_MAF01.ldsc")

Write LD scores

In [150]:
# Initialize output
recantangles = list()

# Get size of LD score matrix
var_stop = bm_LDscore.shape[0]
annot_start = 0
annot_stop = bm_LDscore.shape[1]

# Create list of rectangles
for annot_pos in range(annot_start, annot_stop):
        rectangles.append([0, var_stop, annot_pos, annot_pos + 1])

In [152]:
# Output LD scores
bm_LDscore.export_rectangles(LDSC_dir + "1KG_EUR_MAF01.l2/", rectangles, delimiter = '\t', binary = False)